<a href="https://colab.research.google.com/github/phyop/220505_TensorFlow2/blob/main/220428_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 自然语言处理-词向量模型-Word2Vec

先来考虑第一个问题：如何能将文本向量化呢？看起来比较抽象，可以先从人的角度来观察。

![](https://drive.google.com/uc?export=view&id=1vJ8koE7mnR1eLL34aWc_0NGhH3pXm5ua)

![](https://drive.google.com/uc?export=view&id=1NQGzhXi9pw12_arfWdXA3sL1L9S9jhLL)

![](https://drive.google.com/uc?export=view&id=17MesX_WI_2bv89Mk3rssV6y-Os8jZopO)

![](https://drive.google.com/uc?export=view&id=1bWZYmsxkbUaaSj3fBVhfbe28UnMPK1vT)

![title](https://drive.google.com/uc?export=view&id=1vbAH848_yekyp2SfmNxU6OIF0PR0UjtJ)

![](https://drive.google.com/uc?export=view&id=1d_EK9c9Mr8cXrMPC5p2dK5oc9UfDEb2D)

![title](https://drive.google.com/uc?export=view&id=1T7cdiE2QMWHiTHb2TA3gQw8pzWIyGI3w)

![title](https://drive.google.com/uc?export=view&id=1b0V58LNhJcUlCu5ARkAygMiA9Vso7LAJ)

## 数据从哪来来？

![](https://drive.google.com/uc?export=view&id=14j7DNZ9EcX346YTPkhNqd7dRGt5QFe0w)
![title](https://drive.google.com/uc?export=view&id=1zuimKprh-4bAxX8tEhCgH2DQf2arOoVD)

![title](https://drive.google.com/uc?export=view&id=1uzxNcq-PMrwk6r9W9t4BbT5GEgzq_9v6)

![title](https://drive.google.com/uc?export=view&id=1FXlFKGLkmgqyh654sHiHhyaz8WAtzH1a)

## 不同模型对比

![title](https://drive.google.com/uc?export=view&id=1sip_RjOkXYUykJVcApZrIMAuhzoerqq6)

## Skip-gram模型所需训练数据集 

![title](https://drive.google.com/uc?export=view&id=15kQSieGUJ-M5e-OQu0ov9C3MIX2vWdbu)

![title](https://drive.google.com/uc?export=view&id=1Ww_PJOzElqtPVKmNqIo-al594MG1Z_r3)

## 如何进行训练？ 

![title](https://drive.google.com/uc?export=view&id=1XhJahOlSPFU5x7g5Ot0SairzqdtDGxQ4)

## 如果一个语料库稍微大一些，可能的结果简直太多了，最后一层相当于softmax，计算起来十分耗时，有什么办法来解决嘛？


#### 初始方案：输入两个单词，看他们是不是前后对应的输入和输出，也就相当于一个二分类任务

![title](https://drive.google.com/uc?export=view&id=116sXqw1ofaJVjSTtu3p4RpGKy7eLZ17J)

#### 出发点非常好，但是此时训练集构建出来的标签全为1，无法进行较好的训练

![title](https://drive.google.com/uc?export=view&id=1qTRnJZR30IL8cT0EJCctK94cVTw6w3Nb)

#### 改进方案：加入一些负样本（负采样模型）

![title](https://drive.google.com/uc?export=view&id=1udyLbvAqe0cBINgy6b6ne7ppjH4ZASFZ)

![title](https://drive.google.com/uc?export=view&id=1fi1FzhB_6yQSRQEwIuQLFb6N7wts8Ayx)

## 词向量训练过程

#### 1.初始化词向量矩阵

![title](https://drive.google.com/uc?export=view&id=1cX8WUlGsruMwo3w4XJI6-XyIG-fcPGXO)

![title](https://drive.google.com/uc?export=view&id=1_ehRGZRwz5ThWd4Lg5L3GpR5puddbv4a)

#### 2.通过神经网络返向传播来计算更新，此时不光更新权重参数矩阵W，也会更新输入数据

![title](https://drive.google.com/uc?export=view&id=152Gw1pOBuf2V_dNpak3Q0Rx7cohNiBcl)

In [ ]:
"""
詞空間用50維，可以理解爲，相當於給50個面向去判斷這個詞的傾向。
雖然唐說：「PCM降維完能解釋嗎？就只是計算機能認識它而已」，但盡可能去理解，總還是有辦法解釋的。

要觀察50維的詞向量相似度，看數值是很難看出什麼的，所以要轉成熱度圖，視覺化才有感覺。

word embedding這些50維的向量一開始怎麼構建出來？其實就類似NN一樣。
一開始權重的值是隨機初始化的，然後根據lable去前向傳播計算出loss值，
爲了讓loss值往低方向去移動，所以利用反向傳播，梯度下降法去更新權重值。
（因爲輸入、label都不能改啊，所以可以改的至於權重）

不一樣的是，訓練時NN只改了權重，
但word2vec除了權重，連word embedding這張大表也在更新，
因爲輸入word，需要轉換成vec才能計算，所以改了大表，相當於也改了輸入值。

wx = f
所以在word的反向偏微分更新權重參數外，partial(f)/partial(w)，
還多要反向偏微分更新輸入值，partial(f)/partial(x)。

"Thou shalt not make a machine in the likeness of a human mind"

上面這樣一個句子，如果滑動窗格是3，滑動步數是1。那A、B就是input，C是output。
所以一開始訓練的輸入AB會是“Thou shalt”，輸出C會是“not”。
經過一次滑動後，訓練的輸入的AB會是“shalt not”，輸出C會是“make”。

所以word這樣的訓練模式，不需要自己去做標籤，有文本丟進去，自己滑動窗口就行了。但重點是，word的處理，需要按照“順序”！

# Skipgram，知道中間的詞，去預測上下文
輸入：w(t)
輸出：w(t-2) w(t-1) w(t+1) w(t+2)


# CBOW，知道上下文，去預測中間的詞
輸入：w(t-2) w(t-1) w(t+1) w(t+2)
輸出：w(t)

一般在CBOW中含待填入詞，框到的都是奇數個詞。
因爲比如5個詞好了，第3個詞是待填入，那上、下文各有2個詞。
所以爲了上下文都有相同的詞數，才會框列奇數詞。

word2vec的模型，可以使用gensim工具包。

如果一个语料库稍微大一些，可能的结果简直太多了，最后一层相当于softmax，计算起来十分耗时。
比如語料庫有10萬個詞，那每次要輸出一個詞，就要去對10萬個詞去算softmax幾率，那輸出一段1000字的文本，豈不是要花太～多的計算量了。

初始方案：输入两个单词，看他们是不是前后对应的输入和输出，也就相当于一个二分类任务。
如果可行，就可以從10萬分類，變成2分類而已。
比如原本是輸入Thou，然後經過softmax計算了10萬個詞後，會得到shalt的幾率最高。
但現在變成輸入Thou shalt，用sigmoid的二分類去看，這樣字組的幾率是有多高。

是上下文的，則標記target=1,這是正樣本。
因爲如果是從文本裏面抓，那就都是正樣本，所以每個詞的target就全爲1了。
所以自己創建一些不是訓練集中的詞，比如sfjdoi什麼的，當作negative examples負樣本，標記target=0。這種方法稱爲負採樣模型。

把文本中本來的input、可能的output，都化簡爲連在一起當作input，
然後對這些input都去標記target=1，再對自己亂建的負採樣標記target=0。

有0有1,這些本來需要softmax的，就可以變成是訓練2分類任務了。

而負樣本通常可以選5個，這是因爲去看好用的gensim工具包，裏面的推薦參數是5，所以應該有它的道理。

input 是可查詢的embedding詞匯向量大表，output word是包含正樣本和負樣本。
正向傳播是把input和output樣本連在一起，二分類看sigmoid的幾率高不高，
反向傳播的話，因爲word的處理會連同輸入的詞向量也更改，
而輸入除了embedding詞匯大表以外，還包含了output word的正樣本和負樣本，
所以反向傳播會更新embedding詞匯大表，也會更新output word。
也就是說，負樣本的詞向量也會被更新。

"""